### 오차행렬과 정밀도/재현율

https://velog.io/@ljs7463/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8-%ED%8F%89%EA%B0%80%EC%A0%95%EB%B0%80%EB%8F%84%EC%9E%AC%ED%98%84%EC%9C%A8f1-score%EB%93%B1

![title](images/confusion_matrix.png)

TN : 음성을 음성으로 판별 / 옳은(T) 음성(N) 판별<br>
FP : 음성을 양성으로 판별(오류) / 잘못된(F) 양성(P) 판별<br>
FN : 양성을 음성으로 판별(오류) / 잘못된(F) 음성(F) 판별<br>
TP : 양성을 양성으로 판별 / 옳은(T) 양성(P) 판별<br><br>
정밀도(Precision_score) : TP/(FP+TP) --> 내가 양성(P)으로 판명한 것 중에 실제 양성이 있는 비율. 말 그대로 정확도.<br>
재현율(recall_score/민감도/TPR) : TP/(FN+TP) --> 실제 양성인 것들 중에 내가 양성(P)으로 얼마나 판별해 냈는가의 비율<br>
위양성률(fallout/FPR) : FP/(TN+FP) --> 실제 음성인 것들 중 양성으로 오인, 판별한 것들의 비율<br><br>
코로나 검사를 위해 병원을 찾아온 사람의 검사결과는 재현율이 중요하다.<br>
실제로 코로나인데 코로나가 아니라고 예측하면 큰일나기 때문이다.<br><br>
메일이 왔는데 스팸메일여부를 판단하는 것은 정밀도가 중요하다.<br>
스팸으로 판별한 것들 중 스팸이 아닌 것이 섞여 있어도 큰 문제는 없기 때문이다.<br>
스팸은 보수적으로 접근해 최대한 많이 걸러내는 게 중요하기 때문이다.<br><br>
이렇듯 어떤것에 사용하느냐에 따라 달라지며 대부분은 재현율이 더 중요해 많이 사용하고 있다.<br>

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y, y_pred)) #정밀도와 재현율 출력
print(confusion_matrix(y_test, model.predict(X_test))) #오차행렬 출력

![title](images/f1_score.png)

위 classification_report 결과에서 0이라고 예측한 데이터의 81%만 실제로 0이었고 1이라고 예측한 데이터의 75%만 실제로 1이었음을 알 수 있다. 또한 실제 0인 데이터 중의 84%만 0으로 판별되었고 실제 1인 데이터 중의 71%만 1로 판별되었음을 알 수 있다.<br><br>
다중 클래스 문제의 경우에는 각각의 클래스에 대해 자신을 를 양성 클래스로, 다른 클래스를 음성 클래스로 가정하여 OvR 문제를 풀고 각각에 대해 정밀도, 재현율, 위양성률 등의 평가점수를 구한다. 이렇게 하면 클래스별로 각각 다른 평가점수가 나오므로 이를 하나로 합쳐는 일종의 평균을 구해야 하는데 다음과 같은 여러가지 기준을 사용한다.<br><br>
accuracy: 정확도. 전체 학습데이터의 개수에서 각 클래스에서 자신의 클래스를 정확하게 맞춘 개수의 비율<br>
macro: 단순평균<br>
weighted: 각 클래스에 속하는 표본의 갯수로 가중평균<br>

### F1 Score

F1 Score는 Precision과 Recall의 조화평균으로 주로 분류 클래스 간의 데이터가 불균형이 심각할때 사용한다.<br>
F1 Score는 정밀도와 재현율의 조화평균이다.<br><br>
F1 Score = precision*recall/precision+recall<br><br>
높을수록 좋은 모델이다.

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)

### ROC Curve와 AUC

재현율(recall)과 위양성률(fall-out)은 일반적으로 양의 상관 관계가 있다.<br>
재현율을 높이기 위해서는 양성으로 판단하는 기준(threshold)을 낮추어 약간의 증거만 있어도 양성으로 판단하도록 하면 된다. 그러나 이렇게 되면 음성임에도 양성으로 판단되는 표본 데이터가 같이 증가하게 되어 위양성율이 동시에 증가한다. 반대로 위양성율을 낮추기 위해 양성을 판단하는 기준을 엄격하게 두게 되면 증거 부족으로 음성 판단을 받는 표본 데이터의 수가 같이 증가하므로 재현율이 떨어진다.<br>
이 기준(threshold)의 최적점을 찾기 위해 사용되는 것이 바로 ROC커브와 AUC이다. (ROC : Receiver Operator Characteristic)<br><br>
여기서 말하는 기준, 즉 임계값은 무엇일까?<br>만약 코로나에 걸린 사람을 1 아닌 사람을 0으로 해서 예측을 하는데 있어서 각각의 0과 1은 확률을 가지고 있다. predict(x) 메서드가 아니라 predict_proba(x) 메서드를 쓰면 0과 1로 분류할 확률값을 볼 수 있다. 한 레코드가 0일 확률 0.65, 1일 확률 0.35로 나왔고, 임계치가 0.5로 설정되어 있다면 그 모델은 이 레코드를 0, 음성으로 분류하게 된다. 즉, 임계치는 1(양성)으로 볼 수 있는 기준값이다(양성일 확률에 대한 임계값!). 그런데 이 임계치를 만약 0.3으로 확 낮춰버린다면? 이 레코드의 예측값은 1이 되어 버린다.<br><br>
양성 판별 임계값을 0.3으로 두게 되면 양성일 확률 0.35도 1로 예측한다 즉, 코로나 확진으로 모델은 예측하게 된다. 이는 양성판별 기준이 느슨해 진 것이므로(조금만 의심되어도 양성으로 판별하는 것이므로) 재현율(실제 양성인 것들 중 내가 양성으로 판별한 비율값)을 상승시킨다. 하지만 반대로 정밀도(내가 양성이라고 예측한 것들 중 실제로 양성인 것들의 비율값)는 낮아진다. 또한 FPR(위양성률)도 높이게 된다.<br><br>
따라서 적절한 임계값을 찾아서 정밀도와 재현율을 효율적으로 만들어야한다. 이때 등장하는것이 ROC-curve와 AUC이다. ROC커브는 X축은 FPR(위양성율), Y축은 TPR(재현율)로 재현한 것이며, AUC는 ROC커브의 면적이다.

In [ ]:
rom sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})
roc

![title](images/roc_matrix.png)
![title](images/roc_curve.png)

위 그림을 보면 threshold값이 낮아짐에 따라(양성판별 기준이 느슨해짐에 따라) FPR(위양성율)이 증가하고 동시에 TPR(재현율)도 증가하는 것을 볼 수 있다. 그럼 최적점, 즉 최적의 threshold값을 어디에 둬야 할까?<br><br>

바로 TPR에서 FPR을 뺀 값이 가장 클 때의 threshold를 찾으면 된다. (TPR - FPR)
재현율(실제 양성인데 내가 양성으로 판별한 비율)에서 위양성율(실제 음성인데 내가 양성으로 판별한 비율)을 뺀 값이 가장 클 때, 그 때가 가장 최적점이다.

In [ ]:
# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print('idx:', optimal_idx, ', threshold:', optimal_threshold)

#결과 : idx: 256 , threshold: 0.4633333333333334

AUC값은 ROC곡선 밑의 면적을 구한 것으로, 일반적으로 1에 가까울수록 좋은 수치이고, 0.5에 가까울수록 학습이 제대로 이루어지지 않은 모델을 의미한다. 즉 ROC커브 그래프가 로그함수 모양이면 좋고, 지수함수 모양이면 좋지 않다. AUC가 커지려면 FPR이 작은상태에서 얼마나 큰 TPR을 얻을수 있느냐가 관건이다(=로그함수).즉 가운네 직선에서 멀어지고 왼쪽 상단 모서리쪽으로 가파르게 올라갈 수록 면적이 1에 가까워지는 좋은모델이 된다.<br>
AUC는 최적의 threshold를 찾는데 사용되는 것은 아니고, 모델의 성능을 평가하기 위해 사용되는 것이다.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# predict_proba #0번째 열은 0일 확률, 1번째 열은 1일 확률
predict_proba = model.predict_proba(test_data[x_vars])
acc_score = accuracy_score(test_data["target"], model.predict(test_data[x_vars]))
recall_score = recall_score(test_data["target"], model.predict(test_data[x_vars]))
precision_score = precision_score(test_data["target"], model.predict(test_data[x_vars]))
acc_score, recall_score, precision_score

In [ ]:
#4개 depth를 가진 모델을 가지고 ROC커브로 최적의 threshold를 찾아 정확도를 높여보자.
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])

roc = pd.DataFrame({
    "FPR": fpr, 
    "TPR": tpr, 
    "Threshold": thresholds,
    "TPR-FPR" : tpr-fpr
})
display(roc)
plt.plot(roc["FPR"], roc["TPR"])
maxima = np.argmax(roc["TPR-FPR"])
print("TPR - FPR 최적 threshold값 : {}".format(roc.iloc[maxima, 2]))

In [ ]:
#threshold 값 조정으로 모델 분류성능 높이기
from sklearn.preprocessing import Binarizer

custom_threshold = roc.iloc[maxima, 2]
pred_proba_flatten = model.predict_proba(X_test)[:,1].reshape(-1,1)
binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_flatten) 
custom_predict = binarizer.transform(pred_proba_flatten)
print("오리지날")
print(classification_report(y_test, model.predict(X_test))+"\n\n")

print("threshold : {}".format(custom_threshold))
print(classification_report(y_test, custom_predict)+"\n\n")